In [1]:
import pandas as pd
import csv
import os
import json
import numpy as np
from citipy import citipy
import requests

In [2]:
os.chdir(r"C:\Users\Yanwho\OneDrive\Documents\python-api-challenge")

In [3]:
from config import api_key

In [4]:
lat = np.random.uniform(low=-90.000, high=90.000, size=2000)

In [5]:
long = np.random.uniform(low=-180.000, high=180.000, size=2000)

In [6]:
latlong = zip(lat, long)

In [7]:
coordinates=list(latlong)


In [8]:
cities=[]

In [9]:
# city = citipy.nearest_city(22.99, 120.21)
for x in coordinates:
    city = citipy.nearest_city(x[0],x[1]).city_name
    if city not in cities:
        cities.append(city)
    

In [10]:
# url= "http://api.openweathermap.org/data/2.5/weather?&APPID="+api_key


In [11]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial"
city = []

# Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city
# dont do this for this problem:  query_url=[]


In [23]:
city_name=[]
Latitude=[]
Longitude=[]
Max_Temp=[]
Humidity=[]
Wind_Speed=[]
Cloudiness=[]

In [24]:
city_data=[]


In [25]:
for city in cities:
# the procedure for this loop is to generalize this api request
# query_url = url + "appid=" + api_key + "&q=" + city
    query_url = url + "&appid=" + api_key + "&q=" + city
    weather_response = requests.get(query_url).json()
#     weather_json = weather_response.json()
#     print(query_url)
    try:
        city_name.append(weather_response["name"])
        Latitude.append(weather_response["coord"]["lat"])
        Longitude.append(weather_response["coord"]["lon"])
        Max_Temp.append(weather_response["main"]["temp_max"])
        Humidity.append(weather_response["main"]["humidity"])
        Wind_Speed.append(weather_response["wind"]["speed"])
        Cloudiness.append(weather_response["clouds"]["all"])
        print(city, "success")
    except:
        print("city not found", city)
    pass


bengkulu success
puerto ayora success
rikitea success
albany success
aldan success
palkino success
city not found illoqqortoormiut
mataura success
lerwick success
bredasdorp success
pevek success
city not found tsihombe
ushuaia success
kapaa success
alofi success
srednekolymsk success
tiksi success
nanortalik success
luderitz success
boz success
hobart success
severo-kurilsk success
porbandar success
fort nelson success
arman success
busselton success
city not found taolanaro
kavieng success
lavrentiya success
qaanaaq success
vaini success
rocha success
kyabe success
gamba success
tuktoyaktuk success
umm lajj success
jalu success
victoria success
west bay success
mahebourg success
omsukchan success
damghan success
chake chake success
hasaki success
atar success
cherskiy success
mocuba success
ilulissat success
mahmudabad success
dikson success
binga success
ponta do sol success
punta arenas success
krasnoselkup success
bethel success
hofn success
uzhur success
geraldton success
kadniko

In [26]:
# Get weather data
# weather_response = requests.get(query_url)
# weather_json = weather_response.json()

# Get the temperature from the response
# print(f"The weather API responded with: {weather_json}.")

In [27]:
# instructions for future Ed
# 1. deconstruct every api request output.
# 2. parse important data form the output (E.G. humidity, clouds,etc...)
# 3. append city info to city_data list.  Hint: every itiration (city) within the for loop 
#     is going to be a dictionry with keys being hard coded by me and the values being from the
#     api request output.
# 4. include a try/except clause in the entire for loop where the error is if the city is not found then you "pass" that city

In [30]:
results_df=pd.DataFrame({"City":pd.Series(city_name), 
                         "Latitude":pd.Series(Latitude), 
                         "Longitude":pd.Series(Longitude), 
                         "Max Temp":pd.Series(Max_Temp), 
                         "Humidity":pd.Series(Humidity), 
                         "Wind Speed":pd.Series(Wind_Speed),
                         "Cloudiness":pd.Series(Cloudiness)
                        })

results_df

,City,Latitude,Longitude,Max Temp,Humidity,Wind Speed,Cloudiness
0,Bengkulu,-3.8004,102.2655,75.85,83,4.43,100
1,Puerto Ayora,-0.7393,-90.3518,84.20,58,18.41,40
2,Rikitea,-23.1203,-134.9692,77.90,78,18.05,0
3,Albany,42.6001,-73.9662,34.00,66,3.38,97
4,Aldan,58.6031,125.3894,-28.98,85,2.37,99
...,...,...,...,...,...,...,...
698,Urucará,-2.5364,-57.7600,83.80,80,1.05,71
699,Mosquera,4.7059,-74.2302,68.00,37,10.36,40
700,Fukuechō,32.6881,128.8419,48.69,55,10.47,100
701,Khāsh,28.2211,61.2158,40.55,37,5.26,0


In [32]:
results_df.to_csv("results.csv")